In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time

In [3]:
t0 = time()
df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
# df = pd.read_parquet("../Ishan/1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1000.0 million rows : 93.06386232376099 seconds


In [16]:
df['impressions'].min()

703102

In [17]:
df['impressions'].max()

19938105

In [18]:
df['impressions'].max()/1e6

19.938105

In [4]:
scale = 10_000  # This is put here to stop int32 overflow errors


t0 = time()
impressions = df['impressions'].to_numpy(np.int32) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 49.759377241134644 seconds


In [5]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=750_000/scale, max_val=20e6/scale, entities = data_subjects,ndept=True)
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

what type of child do we have now? [513 513 513 ... 806 806 806] <class 'numpy.ndarray'>
what type of child do we have now? [513 513 513 ... 806 806 806] <class 'numpy.ndarray'>
Time make Private Syft Tensor: 1.1084411144256592 seconds


In [29]:
np.iinfo(np.int64).max/1e18

9.223372036854776

In [23]:
tweets_data.child.max_vals.sum(axis=None)

-1474836480

In [24]:
-1_474_836_480/1e6

-1474.83648

In [6]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into exciting_lecun... done!


In [7]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

Loading dataset... checking asset types...                                                                                                                                    

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft
Uploading Tweets data- 1647980658.8073356: 100%|██| 7/7 [00:08<00:00,  1.26s/it]


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
CPU times: user 39.4 s, sys: 11.7 s, total: 51.1 s
Wall time: 1min 7s


In [8]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1647979291.2153113,Tweets- 1B rows,"[""Tweets data- 1647979291.2153113""] ->",5c7752da-4e8b-49bf-b889-f64c1d6fbcf6
[1],Tweets data- 1647980658.8073356,Tweets- 1B rows,"[""Tweets data- 1647980658.8073356""] ->",7cafe5b5-45cf-4964-91ad-26c4725d754e


In [9]:
data = domain_node.datasets[-1][name]

In [10]:
data

<TensorPointer -> exciting_lecun:8831223f954b455a95fbea9e20689013>

In [11]:
%%time
sum_result = data.sum()
sum_result.block

CPU times: user 2.51 s, sys: 149 ms, total: 2.66 s
Wall time: 1min 44s


<TensorPointer -> exciting_lecun:33a7d3e1cc1d40308ff58a24458b7d1e>

In [12]:
sum_result.exists

True

In [13]:
891896832.0/1e6

891.896832

In [14]:
published_result = sum_result.publish(sigma=1_000_000)

In [15]:
published_result.exists

False

published_result

published_result.request()

domain_node.requests

# domain_node.requests[0].accept()

domain_node.requests

result_of_our_hard_labour = published_result.get()

result_of_our_hard_labour